<a href="https://colab.research.google.com/github/prashantiyaramareddy/AI-ML-Learnings/blob/master/ComputerVision/VGGNet_Implementation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

VGG-Net Implementation

VGGNet is a convolutional neural network architecture proposed by the Visual Geometry Group (VGG) from Oxford University. It is known for its simplicity and uniform architecture, using small convolutional filters and deep layers.

In [9]:
### Import Required Libraries
import torch
import torchvision
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, random_split
from torchvision import datasets, transforms, models
from torch.optim.lr_scheduler import ReduceLROnPlateau


In [10]:
torch.__version__

'2.9.0+cpu'

In [16]:
# Check GPU Availability
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cpu')

### Define transforms to be applied on dataset

In [11]:
from torch.nn.modules import padding

# Use the mean and sta from CIFAR10
mean = (0.4914, 0.4822, 0.4465)
std = (0.2023, 0.1994, 0.2010)

train_transform = torchvision.transforms.Compose([
    torchvision.transforms.RandomCrop(32, padding=4),
    torchvision.transforms.RandomHorizontalFlip(p=0.5),
    transforms.Resize((224, 224)),
    torchvision.transforms.ToTensor(),
    torchvision.transforms.Normalize(mean, std)
])

test_transform = torchvision.transforms.Compose([
    transforms.Resize((224, 224)),
    torchvision.transforms.ToTensor(),
    torchvision.transforms.Normalize(mean, std)
])


In [12]:
# Import Dataset
train_dataset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=train_transform)
test_dataset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=test_transform)



100%|██████████| 170M/170M [00:25<00:00, 6.76MB/s]


In [14]:
# Create DataLoader
val_size = 5000
train_size = len(train_dataset) - val_size
train_dataset, val_dataset = random_split(train_dataset, [train_size, val_size])

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True, pin_memory=True)
val_loader = DataLoader(val_dataset, batch_size=64, shuffle=False, pin_memory=True)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False, pin_memory=True)

In [15]:
len(train_loader), len(test_loader), len(val_loader)

(625, 157, 79)

In [17]:
### Load Pretrained Model
vggnet = models.vgg19(pretrained=True)

# Freeze all layers
for param in vggnet.parameters():
    param.requires_grad = False

# Replace the last fully connected layer
num_classes = 10
vggnet.classifier[6] = nn.Linear(4096, num_classes)

# Only the new final layer should be trainable
for param in vggnet.classifier[6].parameters():
    param.requires_grad = True

vggnet = vggnet.to(device)

/usr/local/lib/python3.12/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG19_Weights.IMAGENET1K_V1`. You can also use `weights=VGG19_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Downloading: "https://download.pytorch.org/models/vgg19-dcbb9e9d.pth" to /root/.cache/torch/hub/checkpoints/vgg19-dcbb9e9d.pth


100%|██████████| 548M/548M [00:09<00:00, 63.6MB/s]


In [20]:
# Train the Model
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(vggnet.parameters(), lr=0.001,
                       weight_decay=5e-4)
scheduler = ReduceLROnPlateau(optimizer, mode='min', factor=0.1, patience=5)

model = vggnet

In [ ]:
from google.colab import output

epochs = 20

for epoch in range(epochs):
  model.train()
  total_EpochLoss = 0
  for images, labels in train_loader:
    images = images.to(device)
    labels = labels.to(device)

    # Forward pass
    outputs = model(images)

    # Calculate loss
    loss = criterion(outputs, labels)

    # Backward pass and optimization
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    total_EpochLoss += loss.item()

avg_loss = total_EpochLoss / len(train_loader)
print(f"Epoch [{epoch+1}/{epochs}], Loss: {avg_loss:.4f}")

model.eval()
val_loss = 0.0

with torch.no_grad():
    for images, labels in val_loader:
        images = images.to(device)
        labels = labels.to(device)

        outputs = model(images)
        loss = criterion(outputs, labels)
        val_loss += loss.item()

avg_val_loss = val_loss / len(val_loader)
print(f"Validation Loss: {avg_val_loss:.4f}")


        outputs = model(images)
        loss = criterion(outputs, labels)